In [1]:
import numpy as np
# Get k-mers from a sequence
def get_kmers(sequence, k):
    return [sequence[x:x+k].lower() for x in range(len(sequence) - k + 1)]

# Get the frequency of each k-mer in a sequence
def freq_kmers(seq, k):
    kmers = get_kmers(seq, k)
    freq = {}
    for kmer in kmers: 
        if kmer not in freq: 
            freq[kmer] = 1
        else: 
            freq[kmer] += 1
    return freq


In [2]:
# Implement Spectrum Kernel
def spectrum_kernel(seq1, seq2, k):
	freq1 = freq_kmers(seq1, k)
	freq2 = freq_kmers(seq2, k)
	for key in freq1:
		if key not in freq2:
			freq2[key] = 0
	for key in freq2:
		if key not in freq1:
			freq1[key] = 0
	freq1 = dict( sorted(freq1.items(), key=lambda x: x[0]) )
	freq2 = dict( sorted(freq2.items(), key=lambda x: x[0]) )
	return np.dot(list(freq1.values()), list(freq2.values()))

In [3]:
# change fasta file into a list of sequences
# fasta reader
def read_fasta(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    return lines

# fasta parser


def parse_fasta(lines):
    seq_list = []
    seq = ''
    seq_name = []
    for line, index in zip(lines, range(len(lines))):
        if index == len(lines) - 1:
            seq += line.strip()
            seq_list.append(seq)
        if line.startswith('>'):
            seq_list.append(seq)
            seq = ''
            name = line.strip()
            seq_name.append(name.split(" /")[0][1:])
            continue
        else:
            seq += line.strip()
    for i in seq_list:
        if i == '':
            seq_list.remove(i)
    return seq_list,seq_name


In [4]:
seq_list, seq_name = parse_fasta(read_fasta('kmeans.fasta'))

In [5]:

print(spectrum_kernel(seq_list[0], seq_list[1], 3))


1951


In [5]:
def diff(key1,key2):
    list1 = list(key1)
    list2 = list(key2)
    dif = 0
    for i in range(len(list1)):
        if list1[i] != list2[i]:
            dif += 1
    return dif

In [6]:
# Implement Mismatch Kernel
def mismatch_kernel(seq1, seq2, k):
	freq1 = freq_kmers(seq1, k)
	freq2 = freq_kmers(seq2, k)
	for key in freq1:
		if key not in freq2:
			freq2[key] = 0
	for key in freq2:
		if key not in freq1:
			freq1[key] = 0
   
	freq1 = dict( sorted(freq1.items(), key=lambda x: x[0]) )
	freq2 = dict( sorted(freq2.items(), key=lambda x: x[0]) )

	newfreq1 = freq1.copy()
	newfreq2 = freq2.copy()

	for key1 in freq1:
		for key2 in freq1:
			dif = diff(key1,key2)
			if dif == 1 and freq1[key2] == 0:
				newfreq1[key1] += 1
			if dif == 1 and freq1[key2] > 0:
				newfreq1[key1] += freq1[key2]
				
	for key1 in freq2:
		for key2 in freq2:
			dif = diff(key1,key2)
			if dif == 1 and freq2[key2] == 0:
				newfreq2[key1] += 1	
			if dif == 1 and freq2[key2] > 0:
				newfreq2[key1] += freq2[key2]

	return np.dot(list(newfreq1.values()), list(newfreq2.values()))

In [7]:
print(mismatch_kernel('AATCCG', 'AATGCC', 2))

54


In [22]:
#for test
seq1 = 'AATCCG'
seq2 = 'AATGCC'
freq1 = freq_kmers(seq1, 2)
freq2 = freq_kmers(seq2, 2)
for key in freq1:
	if key not in freq2:
		freq2[key] = 0
for key in freq2:
	if key not in freq1:
		freq1[key] = 0
   
freq1 = dict( sorted(freq1.items(), key=lambda x: x[0]) )
freq2 = dict( sorted(freq2.items(), key=lambda x: x[0]) )



In [23]:
# test only
newfreq1 = freq1.copy()
newfreq2 = freq2.copy()

for key1 in freq1:
	for key2 in freq1:
		dif = diff(key1,key2)
		if dif == 1 and freq1[key2] == 0:
			newfreq1[key1] += 1
		if dif == 1 and freq1[key2] > 0:
			newfreq1[key1] += freq1[key2]
				
for key1 in freq2:
	for key2 in freq2:
		dif = diff(key1,key2)
		if dif == 1 and freq2[key2] == 0:
			newfreq2[key1] += 1	
		if dif == 1 and freq2[key2] > 0:
			newfreq2[key1] += freq2[key2]	

In [24]:
#test only
newfreq1

{'aa': 2, 'at': 2, 'cc': 4, 'cg': 3, 'gc': 2, 'tc': 4, 'tg': 2}

In [25]:
#test only
newfreq2

{'aa': 2, 'at': 2, 'cc': 4, 'cg': 2, 'gc': 3, 'tc': 3, 'tg': 3}

In [ ]:
#KNN
def dist(seq1, seq2, k):
    return np.sqrt(spectrum_kernel(seq1, seq1, k)-2*spectrum_kernel(seq1, seq2, k)+spectrum_kernel(seq2, seq2, k))

def KNN(train, test, knn_num, kmer_size):
    seq_train = [] 
    seq_name_train = []
    for path in train:
        seq, seq_name = parse_fasta(read_fasta(path))
        seq_train += seq
        seq_name_train += seq_name
    seq_test, seq_name_test = parse_fasta(read_fasta(test))
    train_class_list = ["exons" if "exon" in seq_name_train[i] else "introns" for i in range(len(seq_name_train))]
    test_true_class_list = ["exons" if "exon" in seq_name_test[i] else "introns" for i in range(len(seq_name_test))]
    test_pred_class_list = []
    for i in range(len(seq_test)):
        dist_list = []
        for j in range(len(seq_train)):
            dist_list.append(dist(seq_test[i], seq_train[j], kmer_size))
        dist_list = np.array(dist_list)
        indexes = np.argsort(dist_list)
        exons_num = 0
        introns_num = 0
        for index in indexes[:knn_num]:
            if train_class_list[index] == "exons":
                exons_num += 1
            else:
                introns_num += 1
        if exons_num > introns_num:
            test_pred_class_list.append("exons")
        else:
            test_pred_class_list.append("introns")
    correct = 0
    for i, j in zip( test_true_class_list, test_pred_class_list):
        if i == j:
            correct += 1
    
    accuracy = correct/len(test_true_class_list)

    return accuracy

In [ ]:
KNN(train = ["./KNN/train-exons100.fasta", "./KNN/train-exons100.fasta", "./KNN/train-exons30.fasta", "./KNN/train-introns10.fasta", "./KNN/train-introns30.fasta", "./KNN/train-introns100.fasta"], test = "./KNN/test.fasta", knn_num = 5, kmer_size = 2)

In [6]:
def dist_spectrumkernel(seq1, seq2, k):
    return np.sqrt(spectrum_kernel(seq1, seq1, k)-2*spectrum_kernel(seq1, seq2, k)+spectrum_kernel(seq2, seq2, k))

def dist_mismatchkernel(seq1, seq2, k):
    return np.sqrt(mismatch_kernel(seq1, seq1, k)-2*mismatch_kernel(seq1, seq2, k)+mismatch_kernel(seq2, seq2, k))


In [32]:
#K-Means

def KMeans(seqspath, ncluster, kmer_size, maxniter, distfunc):
    
    seq_list,seq_names = parse_fasta(read_fasta(seqspath))

    

    iter = 0
    
    centroids = np.random.randint(0,len(seq_list),ncluster)
    
    class_list = [ncluster + 1] * len(seq_list)

    while (iter < maxniter):

        
        centroids_new = centroids.copy()

        for i in range(len(seq_list)): #update class for each seq
            dist_list = []
            for j in centroids_new:
                dist_list.append(distfunc(seq_list[i],seq_list[j],kmer_size))
            dist_list = np.array(dist_list)
            class_list[i] = np.argmin(dist_list)

        #update centroids
        for i in range(len(centroids_new)):
            temp_index_list = []
            for j in range(len(class_list)):
                if (class_list[j] == i):
                    temp_index_list.append(j)
            
            sumdis_list = []
            for j in range(len(temp_index_list)):
                sum = 0
                for k in range(len(temp_index_list)):
                    sum += distfunc(seq_list[j],seq_list[k],kmer_size)
                sumdis_list.append(sum)
            
            sumdis_list = np.array(sumdis_list)
        
            centroids_new[i] = temp_index_list[np.argmin(sumdis_list)]

        iter += 1
        if (centroids_new == centroids).all():
            break
        
        centroids = centroids_new.copy()
    
    return centroids, class_list
            


In [33]:
seqspath = "kmeans.fasta"
centroids, class_list =  KMeans(seqspath, 3, 3, 100, dist_spectrumkernel)



In [8]:
seqspath = "kmeans.fasta"

In [8]:
ncluster = 3
kmer_size = 3
maxniter = 1000

seq_list,seq_names = parse_fasta(read_fasta(seqspath))

    #train_class_list = ["exons" if "exon" in seq_name_train[i] else "introns" for i in range(len(seq_name_train))]
    #test_true_class_list = ["exons" if "exon" in seq_name_test[i] else "introns" for i in range(len(seq_name_test))]
    #test_pred_class_list = []

iter = 0
    
centroids = np.random.randint(0,len(seq_list),ncluster)
    
class_list = [ncluster + 1] * len(seq_list)

In [9]:
centroids_new = centroids.copy()

for i in range(len(seq_list)): #update class for each seq
    dist_list = []
    for j in centroids_new:
        dist_list.append(dist_spectrumkernel(seq_list[i],seq_list[j],kmer_size))
    dist_list = np.array(dist_list)
    class_list[i] = np.argmin(dist_list)

In [27]:
for i in range(len(centroids_new)):
    temp_index_list = []
    for j in range(len(class_list)):
        if (class_list[j] == i):
            temp_index_list.append(j)          
    sumdis_list = []
    for j in range(len(temp_index_list)):
        sum = 0
        for k in range(len(temp_index_list)):
            sum += dist_spectrumkernel(seq_list[j],seq_list[k],kmer_size)
        sumdis_list.append(sum)
            
    sumdis_list = np.array(sumdis_list)
        
    centroids_new[i] = temp_index_list[np.argmin(sumdis_list)]


KeyboardInterrupt: 

In [10]:
i = 0
temp_index_list = []
for j in range(len(class_list)):
    if (class_list[j] == i):
        temp_index_list.append(j)          
sumdis_list = []
for j in range(len(temp_index_list)):
    sum = 0
    for k in range(len(temp_index_list)):
        sum += dist_spectrumkernel(seq_list[j],seq_list[k],kmer_size)
    sumdis_list.append(sum)            
sumdis_list = np.array(sumdis_list)
centroids_new[i] = temp_index_list[np.argmin(sumdis_list)]

In [36]:
# Mismatch Kernel for each seq, return a list of kmers+counts
def getmismatchkernel(seq_list, k):
    freq_list = [] #kmers + counts
    allkmers = {}
    for i in range(len(seq_list)):
        freq = freq_kmers(seq_list[i],k)
        freq_list.append(freq)
        for key in freq:
            if key not in allkmers:
               allkmers[key] = key

    for i in range(len(freq_list)):#each dict in freq_list has the union of kmers
        for key in allkmers:
            if key not in freq_list[i]:
                freq_list[i][key] = 0
        freq_list[i] = dict(sorted(freq_list[i].items(), key=lambda x: x[0]))
    freq_list_new = freq_list.copy()  #update occurences with 1 mismatch allowed
    for i in range(len(freq_list_new)):
        for key1 in freq_list_new[i]:
            for key2 in freq_list_new[i]:
                dif = diff(key1,key2)
                if dif == 1 and freq_list[i][key2] == 0:
                    freq_list_new[i][key1] += 1
                if dif == 1 and freq_list[i][key2] > 0:
                    freq_list_new[i][key1] += freq_list[i][key2]
    return freq_list_new


In [16]:
def getdistance(freq1,freq2):
    distance1 = np.dot(list(freq1.values()), list(freq1.values()))
    distance12 = np.dot(list(freq1.values()), list(freq2.values()))
    distance2 = np.dot(list(freq2.values()), list(freq2.values()))
    return np.sqrt(distance1 - 2 * distance12 + distance2)
    
    


In [ ]:
#K-Means

def KMeans_v2_mismatch(seqspath, ncluster, kmer_size, maxniter):
    
    seq_list,seq_names = parse_fasta(read_fasta(seqspath))

    freq_list = getmismatchkernel(seq_list, kmer_size)

    # train_class_list = ["exons" if "exon" in seq_name_train[i] else "introns" for i in range(len(seq_name_train))]
    # test_true_class_list = ["exons" if "exon" in seq_name_test[i] else "introns" for i in range(len(seq_name_test))]
    # test_pred_class_list = []

    iter = 0

    # list: freq of kmers
    

    
    centroids = [freq_list[i] for i in np.random.randint(0,len(seq_list),ncluster)]
    
    class_list = [ncluster + 1] * len(seq_list)

    class_list_new = class_list.copy()
    centroids_new = centroids.copy()

    while (iter < maxniter):

    

        for i in range(len(freq_list)): #update class for each freq
            dist_list = []
            for j in range(len(centroids_new)):
                dist_list.append(getdistance(freq_list[i],centroids_new[j]))
            dist_list = np.array(dist_list)
            class_list_new[i] = np.argmin(dist_list)

        #update centroids
        for i in range(len(centroids_new)):
            flag = True
            for j in range(len(class_list_new)):
                if (flag and class_list_new[j] == i):
                    centroids_new[i] = freq_list[j]
                    flag = False
                if (not flag) and class_list_new[j] == i:
                    for key in centroids_new[i]:
                        centroids_new[i][key] += freq_list[j][key]
            for key in centroids_new[i]:
                centroids_new[i][key] = centroids_new[i][key]/(j + 1)


        iter += 1
        if (class_list_new == class_list).all():
            break
        
        class_list = class_list_new.copy()
    
    return class_list
            

           
                    
            

            

    
   

In [37]:
seq_list,seq_names = parse_fasta(read_fasta(seqspath))
kmer_size = 3
freq_list = getmismatchkernel(seq_list, kmer_size)


In [41]:
iter = 0
ncluster = 3
centroids = [freq_list[i] for i in np.random.randint(0,len(seq_list),ncluster)]
    
class_list = [ncluster + 1] * len(seq_list)

class_list_new = class_list.copy()
centroids_new = centroids.copy()


In [51]:
for i in range(len(freq_list)): #update class for each freq
    dist_list = []
    for j in range(len(centroids_new)):
        dist_list.append(getdistance(freq_list[i],centroids_new[j]))
    dist_list = np.array(dist_list)
    class_list_new[i] = np.argmin(dist_list)


In [53]:
for i in range(len(centroids_new)):
    flag = True
    for j in range(len(class_list_new)):
        if (flag and class_list_new[j] == i):
            centroids_new[i] = freq_list[j]
            flag = False
        if (not flag) and class_list_new[j] == i:
            for key in centroids_new[i]:
                centroids_new[i][key] += freq_list[j][key]
    for key in centroids_new[i]:
        centroids_new[i][key] = centroids_new[i][key]/(j + 1)


In [57]:
iter += 1
(class_list_new == class_list)

        
class_list = class_list_new.copy()